In [1]:
import pandas as pd
import numpy as np
import pickle
import ast
import os
import copy
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.graph_objects as go
import random

from PIL import Image
from transformers import pipeline
from wordcloud import WordCloud
from matplotlib.colors import ListedColormap
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from bertopic import BERTopic
from umap import UMAP
from typing import List, Union
from sklearn.preprocessing import normalize

torch.cuda.is_available()

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/

True

In [2]:
# Read the excel review file
df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_labelled.csv")
df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_sentiment_labelled.csv")
df3 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_emotions_labelled.csv")

# Create a processed_data to be iso with the preprocessing output from the OOP 
df3["processed_data"] = df3["allComment"]
# Convert the string values in the 'predicted_labels' column into lists
df3['predicted_labels'] = df3['predicted_labels'].apply(lambda x: ast.literal_eval(x))
# Convert the string values in the 'predicted_scores' column into dictionaries
df3['predicted_scores'] = df3['predicted_scores'].apply(lambda x: ast.literal_eval(x))
# Convert the string values in the 'predicted_scores' column into dictionaries
df3['proba_dict'] = df3['proba_dict'].apply(lambda x: ast.literal_eval(x))
# Fill empty values in Market Segment column
df3['Market Segment'] = df3['Market Segment'].fillna('Unknown')
# Convertir la colonne de date en un objet datetime
df3['Response Date'] = pd.to_datetime(df3['Response Date'])
# Extraire l'année et le mois et les stocker dans une nouvelle colonne
df3['year_month'] = df3['Response Date'].dt.to_period('M').dt.to_timestamp()


df4 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv")

/tmp/ipykernel_8627/2415188386.py:2: DtypeWarning: Columns (6,7,11,15,16,17,19,30,31,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_labelled.csv")
/tmp/ipykernel_8627/2415188386.py:3: DtypeWarning: Columns (7,8,12,16,17,18,20,31,32,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_sentiment_labelled.csv")
/tmp/ipykernel_8627/2415188386.py:4: DtypeWarning: Columns (8,9,13,17,18,19,21,32,33,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_emotions_labelled.csv")
/tmp/ipykernel_8627/2415188386.py:22: DtypeWarning: Columns (9,10,14,18,19,20,22,33,34,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv("/media/

In [ ]:
### Analyse de Sentiments simple : positif - négatif - neutre

def load_model_huggingface(model_name, task, problem_type=None, **kwargs):
    """
    This function loads a model and tokenizer from a given model name, then creates a pipeline to perform a specified task.

    Args:
        model_name (str): The name of the model to load.
        task (str): The type of task to perform with the pipeline.
        problem_type (str): The type of problem to solve ("multi_label_classification" for multi-label tasks).
        **kwargs: Additional arguments to pass to the pipeline.

    Returns:
        pipeline: A pipeline configured to perform the specified task with the loaded model and tokenizer.
    """
    model = AutoModelForSequenceClassification.from_pretrained(model_name, problem_type=problem_type)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline(task, model=model, tokenizer=tokenizer, **kwargs)
    return classifier

def add_single_label_predictions(df, predictions, predicted_column_name):
    """
    This function merges the DataFrame of single-label predictions with the original DataFrame.

    Args:
        df (pd.DataFrame): The original DataFrame.
        predictions (list): The list of predictions. Each prediction is a dictionary containing a 'label' and a 'score'.
        predicted_column_name (str): The name of the column to be added to the DataFrame.

    Returns:
        pd.DataFrame: The original DataFrame with added columns for the predicted labels and their scores.
    """
    predicted_df = df
    # Convert the predictions to a DataFrame
    prediction_results = pd.DataFrame(predictions)
    prediction_results.rename(columns={'label': predicted_column_name}, inplace=True)
    # # Reset the indices of the DataFrames (if necessary)
    # df.reset_index(drop=True, inplace=True)
    # prediction_results.reset_index(drop=True, inplace=True)
    # Merge the original DataFrame with the prediction results
    df_predicted = pd.concat([predicted_df, prediction_results], axis=1)
    return df_predicted

def add_multi_label_predictions(df, predictions, predicted_column_name):
    """
    This function adds a new column with multi-label predictions to the DataFrame and also adds two more columns for 
    the best label and its score.

    Args:
        df (pd.DataFrame): The original DataFrame.
        predictions (list): The list of predictions. Each prediction is a list of dictionaries, where each dictionary 
                            contains a 'label' and a 'score'.
        predicted_column_name (str): The name of the column to be added to the DataFrame.

    Returns:
        pd.DataFrame: The original DataFrame with added columns for the predicted labels and their scores, as well as 
                      columns for the best label and its score.
    """
    predicted_df = df
    # Keep the original predictions as they are (a list of dictionaries) and add them to the DataFrame as a new column
    predicted_df[predicted_column_name] = predictions
    # Add columns for the best label and its score
    predicted_df[f'best_{predicted_column_name}'] = predicted_df[predicted_column_name].apply(lambda x: max(x.keys(), key=lambda k: x[k]) if x else None)
    predicted_df[f'best_{predicted_column_name}_score'] = predicted_df[predicted_column_name].apply(lambda x: x[max(x.keys(), key=lambda k: x[k])] if x else None)
    return predicted_df

def make_predictions_df(classifier, df, predicted_column_name):
    """
    This function makes predictions on a DataFrame of documents using a given classifier. It adds the predictions to 
    the DataFrame as new columns. If the classifier is for single-label classification, it adds one column for the 
    predicted label and one for the score. If the classifier is for multi-label classification, it adds one column 
    with a dictionary of label-score pairs for each document, and two additional columns for the best label and its score.

    Args:
        classifier (pipeline): The Hugging Face pipeline object for making predictions.
        df (pd.DataFrame): The DataFrame containing the documents to make predictions on. It must have a 'processed_data' 
                           column with the preprocessed text of each document.
        predicted_column_name (str): The name of the column to be added to the DataFrame for the predictions.

    Returns:
        pd.DataFrame: The original DataFrame with added columns for the predictions.
    """
    # Get the list of documents from the DataFrame
    docs = df["processed_data"].tolist()
    # Get predictions
    predictions = classifier(docs)
    
    # Check if predictions is a list of dictionaries (single-label case)
    if isinstance(predictions, list) and isinstance(predictions[0], dict):
        df_predicted = add_single_label_predictions(df, predictions, predicted_column_name)
    
    # Multi-label case
    elif isinstance(predictions, list) and isinstance(predictions[0], list):
        df_predicted = add_multi_label_predictions(df, predictions, predicted_column_name)

    return df_predicted

classifier = load_model_huggingface("cardiffnlp/twitter-roberta-base-sentiment-latest", "text-classification", max_length=512, truncation=True)
predicted_df = make_predictions_df(classifier, df4, 'sentiment_label')

# model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
# classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, max_length=512, truncation=True)

# predictions = []
# for review in [[x] for x in df.allComment.tolist()]:
      ## Effectuer une prédiction pour le document en utilisant le classificateur
#     prediction = classifier(review)
      ## Ajouter la prédiction à la liste des prédictions
#     predictions.append(prediction)

## Création d'un DataFrame à partir des résultats de prédiction
# res = pd.DataFrame([item for sublist in predictions for item in sublist])
# df_pred = pd.concat([df, res], axis=1)

# The model created a column 'label' to store the prediction, BUT the df already had one column 'label', then :
# # Rename the second 'label' column to 'sentiment_label'
# cols = df_pred.columns.tolist()
# cols[len(cols) - 1 - cols[::-1].index('label')] = 'sentiment_label'
# df_pred.columns = cols

# df_pred.to_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_sentiment_labelled.csv")

In [ ]:
### Analyse de Sentiments approfondie : par 28 émotions

# # en multilabel, avec GPU et par batch pour accélerer le traitement

# # Check if a CUDA-enabled GPU is available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model_name = "SamLowe/roberta-base-go_emotions"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, problem_type="multi_label_classification", max_length=512)

# # Move the model to the GPU
# model = model.to(device)

# # Set the batch size
# batch_size = 2

# # Create a list of label names
# label_emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise','neutral']

# # Initialize lists to store the predicted labels and scores
# predicted_labels = []
# predicted_scores = []

# df3 = df2
# # Iterate over the rows of the DataFrame in batches
# for i in range(0, len(df3), batch_size):
#     batch = df3[i:i+batch_size]
#     texts = batch['allComment'].tolist()
#     inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    
#     # Move the inputs to the GPU
#     inputs = {k: v.to(device) for k, v in inputs.items()}
    
#     outputs = model(**inputs)
#     probs = outputs.logits.sigmoid().detach().cpu().numpy()
    
#     # Apply a threshold to the probabilities to get the predicted labels
#     threshold = 0.5
#     labels = [[label_emotions[i] for i, prob in enumerate(prob_row) if prob > threshold] for prob_row in probs]
    
#     # Store the predicted labels and scores
#     predicted_labels.extend(labels)
#     scores = [{label_emotions[i]: prob for i, prob in enumerate(prob_row)} for prob_row in probs]
#     # predicted_scores.extend(probs.tolist())
#     predicted_scores.extend(scores)

# # Add the predicted labels and scores as new columns in the DataFrame
# df3['predicted_labels'] = predicted_labels
# df3['predicted_scores'] = predicted_scores

# # Convert the string values in the 'predicted_labels' column into lists
# df3['predicted_labels'] = df3['predicted_labels'].apply(lambda x: ast.literal_eval(x))
# # Convert the string values in the 'predicted_scores' column into dictionaries
# df3['predicted_scores'] = df3['predicted_scores'].apply(lambda x: ast.literal_eval(x))

# # df3.to_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_emotions_labelled.csv")

In [ ]:
def convert_dict(input_dict, conversion_type='keys_to_values'):
    """
    This function converts between a key-to-value dictionary and a value-to-keys dictionary.

    Args:
        input_dict (dict): The input dictionary. If conversion_type is 'keys_to_values', this should be a dictionary where 
                           the keys are the original keys and the values are the corresponding values. If conversion_type is 
                           'values_to_keys', this should be a dictionary where the keys are the original values and the values 
                           are lists of keys for each value.
        conversion_type (str): The type of conversion to perform. Can be either 'keys_to_values' or 'values_to_keys'.

    Returns:
        dict: The converted dictionary. If conversion_type is 'keys_to_values', this will be a dictionary where the keys 
              are the original values and the values are lists of keys for each value. If conversion_type is 'values_to_keys', 
              this will be a dictionary where the keys are the original keys and the values are the corresponding values.
    """
    if conversion_type == 'keys_to_values':
        output_dict = {}
        for key, value in input_dict.items():
            if value not in output_dict:
                output_dict[value] = []
            output_dict[value].append(key)
    elif conversion_type == 'values_to_keys':
        output_dict = {key: value for value, keys in input_dict.items() for key in keys}
    else:
        raise ValueError("Invalid conversion_type. Must be either 'keys_to_values' or 'values_to_keys'.")
    
    return output_dict

def add_grouping_column(df, key_column, group_dict, group_column_name):
    """
    This function adds a new column to a DataFrame with the group name of each key.

    Args:
        df (pd.DataFrame): The original DataFrame.
        key_column (str): The name of the column in df that contains the keys.
        group_dict (dict): A dictionary where the keys are the original keys and the values are the corresponding group names.
        group_column_name (str): The name of the new column to be added to df for the groups.

    Returns:
        pd.DataFrame: The original DataFrame with an added column for the groups.
    """
    df[group_column_name] = df[key_column].map(group_dict)
    return df

# transform the multilabel results from the emotions analysis in single label
# Define the groups
positive_emotions = ['admiration','approval','gratitude','caring','realization','joy','optimism','love','excitement','amusement','relief']
negative_emotions = ['disappointment','disapproval','annoyance','confusion','nervousness','fear','sadness','remorse','disgust','embarrassment','anger']
neutral_emotions = ['neutral','desire','surprise','curiosity']
# Create a dictionary where the keys are the group names and the values are the lists of labels
groups = {"positive": positive_emotions, "negative": negative_emotions, "neutral": neutral_emotions}
# Convert the groups to a group_dict
group_dict = convert_dict(groups, conversion_type='values_to_keys')

# Now you can use group_dict with add_grouping_column
df3 = add_grouping_column(df3, "predicted_label", group_dict, "sentiment_from_emotion_label")

# # Create a new column 'single_emotion_label' that contains the label with the highest score
# df3['single_emotion_label'] = df3['predicted_scores'].apply(lambda x: max(x, key=x.get))
# # Create a new column 'sentiment' that contains the sentiment of the emotion in the 'single_emotion_label' column
# df3['single_sentiment_from_emotion'] = df3['single_emotion_label'].apply(lambda x: 'positive' if x in positive_emotions else ('negative' if x in negative_emotions else ('neutral' if x in neutral_emotions else 'unknown')))

# df3.to_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv")

# Use the explode() method to transform each element of the list into a row
# new_df3 = df3.explode('predicted_labels')
# new_df3 = new_df3.rename(columns={'predicted_labels': 'emotion_label'})

# Pour retourner au df initial avec une liste de label
# Group the rows by the original index and aggregate the 'emotion_label' column into a list
# original_df = new_df.groupby(new_df.index).agg({'emotion_label': list})

In [ ]:
### comparaison entre analyse de sentiment simple et les résultats du single label issus de l'analyse d'émotions

# Calculate the total number of rows for each sentiment in the 'sentiment_label' column
total_positive = len(df3[df3['sentiment_label'] == 'positive'])
total_negative = len(df3[df3['sentiment_label'] == 'negative'])
total_neutral = len(df3[df3['sentiment_label'] == 'neutral'])

# Calculate the number of matches for each sentiment
positive_matches = len(df3[(df3['sentiment_label'] == 'positive') & (df3['single_sentiment_from_emotion'] == 'positive')])
negative_matches = len(df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_from_emotion'] == 'negative')])
neutral_matches = len(df3[(df3['sentiment_label'] == 'neutral') & (df3['single_sentiment_from_emotion'] == 'neutral')])

# Calculate the percentage of matches for each sentiment
positive_match_percent = positive_matches / total_positive * 100 if total_positive > 0 else 0
negative_match_percent = negative_matches / total_negative * 100 if total_negative > 0 else 0
neutral_match_percent = neutral_matches / total_neutral * 100 if total_neutral > 0 else 0

# Calculate the number of differences for each sentiment
positive_differences = total_positive - positive_matches
negative_differences = total_negative - negative_matches
neutral_differences = total_neutral - neutral_matches

# Calculate the percentage of differences for each sentiment
positive_difference_percent = positive_differences / total_positive * 100 if total_positive > 0 else 0
negative_difference_percent = negative_differences / total_negative * 100 if total_negative > 0 else 0
neutral_difference_percent = neutral_differences / total_neutral * 100 if total_neutral > 0 else 0

# Calculate the distribution of differences for each sentiment
positive_to_negative = len(df3[(df3['sentiment_label'] == 'positive') & (df3['single_sentiment_from_emotion'] == 'negative')])
positive_to_neutral = len(df3[(df3['sentiment_label'] == 'positive') & (df3['single_sentiment_from_emotion'] == 'neutral')])
negative_to_positive = len(df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_from_emotion'] == 'positive')])
negative_to_neutral = len(df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_from_emotion'] == 'neutral')])
neutral_to_positive = len(df3[(df3['sentiment_label'] == 'neutral') & (df3['single_sentiment_from_emotion'] == 'positive')])
neutral_to_negative = len(df3[(df3['sentiment_label'] == 'neutral') & (df3['single_sentiment_from_emotion'] == 'negative')])

# Print the results
print(f"Positive matches: {positive_match_percent:.2f}%")
print(f"Negative matches: {negative_match_percent:.2f}%")
print(f"Neutral matches: {neutral_match_percent:.2f}%")
print(f"Positive differences: {positive_difference_percent:.2f}%")
print(f"Negative differences: {negative_difference_percent:.2f}%")
print(f"Neutral differences: {neutral_difference_percent:.2f}%")
print(f"Positive to Negative: {positive_to_negative / total_positive * 100:.2f}%" if total_positive > 0 else "Positive to Negative: N/A")
print(f"Positive to Neutral: {positive_to_neutral / total_positive * 100:.2f}%" if total_positive > 0 else "Positive to Neutral: N/A")
print(f"Negative to Positive: {negative_to_positive / total_negative * 100:.2f}%" if total_negative > 0 else "Negative to Positive: N/A")
print(f"Negative to Neutral: {negative_to_neutral / total_negative * 100:.2f}%" if total_negative > 0 else "Negative to Neutral: N/A")
print(f"Neutral to Positive: {neutral_to_positive / total_neutral * 100:.2f}%" if total_neutral > 0 else "Neutral to Positive: N/A")
print(f"Neutral to Negative: {neutral_to_negative / total_neutral * 100:.2f}%" if total_neutral > 0 else "Neutral to Negative: N/A")

# Filter the DataFrame to only include rows where the 'sentiment_label' is 'negative' and the 'single_sentiment_label' is 'neutral'
negative_to_neutral = df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_from_emotion'] == 'neutral')]

# Calculate the distribution of emotions in the 'single_emotion_label' column
emotion_distribution = negative_to_neutral['single_emotion_label'].value_counts(normalize=True) * 100

# Print the results
emotion_distribution

In [3]:
### chargement du model bertopic

def load_bertopic_model(filename):
    """
    Load a BERTopic model and associated data from a file.
    
    :param filename: The name of the file to load the data from.
    :return: A tuple containing the loaded BERTopic model, topics, probs, and docs variables.
    """
    # Load the BERTopic model
    topic_model = BERTopic.load(filename)
    
    # Load the topics, probs, and docs variables
    with open(filename + '_data.pkl', 'rb') as f:
        topics, probs, embeddings, docs = pickle.load(f)
    
    return topic_model, topics, probs, embeddings, docs

topic_model, topics, probs, embeddings, docs = load_bertopic_model('../models/raw_keybert_bertopic_model')

In [4]:
### Création du modèle bertopic aggrégé pour topics finaux

def create_merged_model(docs, bertopic_model, topics_to_merge_dict, label_names_dict):
    """
    Create a new BERTopic model by merging topics from an existing model.

    This function takes as input a list of documents `docs`, an existing BERTopic model `bertopic_model`, a dictionary `topics_to_merge_dict` specifying which topics to merge, and a dictionary `label_names_dict` specifying the labels for the merged topics.

    The function creates a deep copy of the input BERTopic model and merges the specified topics using the `merge_topics` method. Then, it sets the topic labels for the merged model using the `set_topic_labels` method and the provided `label_names_dict`.

    The resulting merged BERTopic model is then returned.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The input BERTopic model to be merged.
        topics_to_merge_dict (dict): A dictionary specifying which topics to merge. The keys are the topic numbers to be merged, and the values are the topic numbers into which they should be merged.
        label_names_dict (dict): A dictionary specifying the labels for the merged topics. The keys are the topic numbers, and the values are the corresponding labels.

    Returns:
        BERTopic: The resulting merged BERTopic model.
    """
    topic_model_merged = copy.deepcopy(bertopic_model)
    topic_model_merged.merge_topics(docs, topics_to_merge_dict)

    # Create a dictionary to match the aggregated name to their corresponding topic number
    mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_dict)}
    # Set topic labels for the aggregated model
    topic_model_merged.set_topic_labels(mergedtopic_labels_dict)

    return topic_model_merged

# List of topics numbers. Each value of this list is a list that contains the topic number of the topics to join together
topics_to_merge = [ [42,3,0,13], #Delivery Deadlines : challenges and strategies involved in managing delivery deadlines in logistics operations. (vert)
                    [20,50,27], #Quotation and Pricing Strategies (vert bas)
                    [35,32], #Touch Panels and Screens (rouge, haut)
                    [40,36], #Frequency Converters : frequency converters used in industrial applications and the technical support provided by manufacturers and suppliers (rouge, suite)
                    [37,21,6,12,9,4,1,14,16,31,19], #“Automation Components” : hardware and software components used in industrial automation systems. (rouge centre)
                    [33,46,8], #Product Evaluation : evaluate the quality, affordability and reliability of products and services (rouge, fin)
                    [44,51,23,41,49,57,22], #Customer Support : Reliability and Quality in Customer Service and Support (bleu ciel)
                    [58,59], #Quick Customer Service (marron)
                    [38,10,26,52,39,43], #Problem Solving and Communication (focus on the importance of being efficient and precise when solving problems) (jaune)
                    [45,47,55,53,54], #Assistance and Guidance (noir)
                    [29,30,11,24], #Power Supply Issues (2e vert, haut)
                    [7,5,2,25,15,34,18,28,17], #Technical Support (2e vert, bas)
                    [48,56] #None : positive feedback (2e rouge)
]

# Set the topic names for the new aggregated topic
# It must match the order from the topics_to_merge list
label_names = [
    "Outliers",
    "Automation Components",
    "Technical Support",
    "Delivery Deadlines",
    "Problem Solving & Comm",
    "Power Supply Issues",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Product Evaluation",
    "Pricing", #Quotation and Pricing Strategies
    "Assistance", #Assistance and Guidance
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Positive feedback",
    "Quick Customer Service"
    ]

# Create a new merged bertopic model 
topic_model_merged = create_merged_model(docs, topic_model, topics_to_merge, label_names)
# topic_model_merged.get_topic_info()

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


In [7]:
# # Modify the visualize_topics_per_class method from bertopic to be able to print a barchart vertically
# def visualize_topics_per_class_orient(topic_model,
#                                topics_per_class: pd.DataFrame,
#                                top_n_topics: int = 10,
#                                topics: List[int] = None,
#                                normalize_frequency: bool = False,
#                                custom_labels: Union[bool, str] = False,
#                                title: str = "<b>Topics per Class</b>",
#                                width: int = 1250,
#                                height: int = 900,
#                                orient: str = "h") -> go.Figure:
#     """ Visualize topics per class

#     Arguments:
#         topic_model: A fitted BERTopic instance.
#         topics_per_class: The topics you would like to be visualized with the
#                           corresponding topic representation
#         top_n_topics: To visualize the most frequent topics instead of all
#         topics: Select which topics you would like to be visualized
#         normalize_frequency: Whether to normalize each topic's frequency individually
#         custom_labels: If bool, whether to use custom topic labels that were defined using 
#                        `topic_model.set_topic_labels`.
#                        If `str`, it uses labels from other aspects, e.g., "Aspect1".
#         title: Title of the plot.
#         width: The width of the figure.
#         height: The height of the figure.
#         orient: The orientation of the barchart, 'h' for horizontal, anything else for vertical

#     Returns:
#         A plotly.graph_objects.Figure including all traces

#     Examples:

#     To visualize the topics per class, simply run:

#     ```python
#     topics_per_class = topic_model.topics_per_class(docs, classes)
#     topic_model.visualize_topics_per_class(topics_per_class)
#     ```

#     Or if you want to save the resulting figure:

#     ```python
#     fig = topic_model.visualize_topics_per_class(topics_per_class)
#     fig.write_html("path/to/file.html")
#     ```
#     <iframe src="../../getting_started/visualization/topics_per_class.html"
#     style="width:1400px; height: 1000px; border: 0px;""></iframe>
#     """
#     colors = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#D55E00", "#0072B2", "#CC79A7"]

#     # Select topics based on top_n and topics args
#     freq_df = topic_model.get_topic_freq()
#     freq_df = freq_df.loc[freq_df.Topic != -1, :]
#     if topics is not None:
#         selected_topics = list(topics)
#     elif top_n_topics is not None:
#         selected_topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
#     else:
#         selected_topics = sorted(freq_df.Topic.to_list())

#     # Prepare data
#     if isinstance(custom_labels, str):
#         topic_names = [[[str(topic), None]] + topic_model.topic_aspects_[custom_labels][topic] for topic in topics]
#         topic_names = ["_".join([label[0] for label in labels[:4]]) for labels in topic_names]
#         topic_names = [label if len(label) < 30 else label[:27] + "..." for label in topic_names]
#         topic_names = {key: topic_names[index] for index, key in enumerate(topic_model.topic_labels_.keys())}
#     elif topic_model.custom_labels_ is not None and custom_labels:
#         topic_names = {key: topic_model.custom_labels_[key + topic_model._outliers] for key, _ in topic_model.topic_labels_.items()}
#     else:
#         topic_names = {key: value[:40] + "..." if len(value) > 40 else value
#                        for key, value in topic_model.topic_labels_.items()}
#     topics_per_class["Name"] = topics_per_class.Topic.map(topic_names)
#     data = topics_per_class.loc[topics_per_class.Topic.isin(selected_topics), :]

#     # Add traces
#     fig = go.Figure()
#     for index, topic in enumerate(selected_topics):
#         if index == 0:
#             visible = True
#         else:
#             visible = "legendonly"
#         trace_data = data.loc[data.Topic == topic, :]
#         topic_name = trace_data.Name.values[0]
#         words = trace_data.Words.values
#         if normalize_frequency:
#             x = normalize(trace_data.Frequency.values.reshape(1, -1))[0]
#         else:
#             x = trace_data.Frequency

#         ### Old part from the source github of bertopic ###
#         # fig.add_trace(go.Bar(y=trace_data.Class,
#         #                      x=x,
#         #                      visible=visible,
#         #                      marker_color=colors[index % 7],
#         #                      hoverinfo="text",
#         #                      name=topic_name,
#         #                      orientation="h",
#         #                      hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))
        
#         fig.add_trace(go.Bar(y=trace_data.Class if orient == "h" else x,
#                             x=x if orient == "h" else trace_data.Class,
#                             visible=visible,
#                             marker_color=colors[index % 7],
#                             hoverinfo="text",
#                             name=topic_name,
#                             orientation=orient,
#                             hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))

#     # Styling of the visualization
#     fig.update_xaxes(showgrid=True)
#     fig.update_yaxes(showgrid=True)
#     fig.update_layout(
#         xaxis_title="Normalized Frequency" if normalize_frequency else "Frequency",
#         yaxis_title="Class",
#         title={
#             'text': f"{title}",
#             'y': .95,
#             'x': 0.40,
#             'xanchor': 'center',
#             'yanchor': 'top',
#             'font': dict(
#                 size=22,
#                 color="Black")
#         },
#         template="simple_white",
#         width=width,
#         height=height,
#         hoverlabel=dict(
#             bgcolor="white",
#             font_size=16,
#             font_family="Rockwell"
#         ),
#         legend=dict(
#             title="<b>Global Topic Representation",
#         )
#     )
#     return fig

In [158]:
def create_topics_per_class_df(df, bertopic_model, classes_column, filter=False, filter_group=None, filter_value=None,  sortedBy=None, ascending=True):
    """
    Create a dataframe representing the topics per class.

    This function takes as input a dataframe `df`, a BERTopic model `bertopic_model`, a column name `classes_column` representing the classes, an optional boolean parameter `filter` which determines whether to filter the data based on a subclass, an optional parameter `filter_group` representing the column name of the subclass to filter on, an optional parameter `filter_value` representing the value of the subclass to filter on, an optional parameter `sortedBy` which can be used to sort the topics by either "Frequency" or "Name", and an optional boolean parameter `ascending` which determines the sorting order (ascending or descending).

    The function first checks that the values of `sortedBy`, `ascending`, and `subclass_name` and `subclass_value` (if `filter` is `True`) are valid. If any of these values are not valid, a ValueError is raised with an appropriate error message.

    Then, depending on whether `filter` is `True` or not, the function either calculates the topics per class using the `topics_per_class` method of the BERTopic model or calls a nested function `topics_per_subclass` to compute the topics per class with filtering.

    If `sortedBy` is "Frequency", the resulting dataframe is sorted by frequency in either ascending or descending order depending on the value of `ascending`.

    The resulting dataframe is then returned.

    Parameters:
        df (pandas.DataFrame): The input dataframe containing the data to be used.
        bertopic_model (BERTopic): The BERTopic model used to calculate the topics per class.
        classes_column (str): The name of the column in `df` representing the classes.
        filter (bool): An optional boolean parameter used to determine whether to filter the data based on a subclass. Defaults to False.
        filter_group (str): An optional parameter representing the column name of the subclass to filter on. Only used if `filter` is True. Defaults to None.
        filter_value: An optional parameter representing the value of the subclass to filter on. Only used if `filter` is True. Defaults to None.
        sortedBy (str): An optional parameter used to sort the topics by either "Frequency" or "Name". Defaults to None.
        ascending (bool): An optional boolean parameter used to determine the sorting order. If True, sorts in ascending order. If False, sorts in descending order. Defaults to True.

    Returns:
        pandas.DataFrame: A dataframe representing the topics per class.
    """
    
    # Check that the value of sortedBy is valid
    if sortedBy not in [None, "Frequency", "Name"]:
        raise ValueError("sortedBy must be either None (default value), 'Frequency', or 'Name'")
    # Check that ascending is only used if sortedBy is not None
    if sortedBy is None and ascending != True:
        raise ValueError("ascending can only be used if sortedBy parameter is used")
    # Check that the value of ascending is valid
    if ascending not in [True, False]:
        raise ValueError("ascending must be either True or False")
    # Check that subclass_name and subclass_value are provided if filter is True
    if filter and (filter_group is None or filter_value is None):
        raise ValueError("If filter is True, both subclass_name and subclass_value must be provided")
    
    # Define a function to compute topics_per_class with filtering
    def topics_per_subclass():
        """
        Create a dataframe that contains the topic number, the list of words that describe the topic,
        and the frequency of documents from this topic that belong to the element from the first "Class" column
        for a subset of data that is filtered by a given subclass value.
        Basically it does the same as topics_per_class method from bertopic adding a filter that depends on an other class

        :param df: A pandas DataFrame containing the full data
        :param bertopic_model: A bertopic model used to compute the topics
        :param classes_column: The name of the column in df that contains the class values
        :param filter_group: The name of the column in df that contains the subclass values (the filter values)
        :param filter_value: The value of the subclass to filter the data by
        :return: A pandas DataFrame containing the topic number, the list of words that describe the topic,
                and the frequency of documents from this topic that belong to the element from the first "Class" column
                for the filtered data (subclass data)
        """
        # Filter your data based on the values from the chosen subclass
        filtered_data = df[df[filter_group] == filter_value]
        classes_filtered_data=filtered_data[classes_column].astype(str).tolist()
        filtered_topics = [bertopic_model.topics_[i] for i in filtered_data.index.tolist()]

        # Create manually a topic_per_class dataframe from a subset of the full documents
        topics_per_subClass_df = pd.DataFrame({'Topic': filtered_topics, 'Class': classes_filtered_data})
        # Calculate the frequency of each topic for each class
        topics_per_subClass_df = topics_per_subClass_df.groupby(['Topic', 'Class']).size().reset_index(name='Frequency')
        # Add the words that describe each topic
        topic_words = {row['Topic']: row['Name'] for _, row in bertopic_model.get_topic_info().iterrows()}
        topics_per_subClass_df['Words'] = topics_per_subClass_df['Topic'].map(topic_words)

        # Add rows for missing topics with a frequency of 0
        missing_topics = set(bertopic_model.get_topics().keys()) - set(topics_per_subClass_df['Topic'].unique())
        for topic in missing_topics:
            for class_ in topics_per_subClass_df['Class'].unique():
                new_row = pd.DataFrame({
                    'Topic': [topic],
                    'Words': [topic_words[topic]],
                    'Frequency': [0],
                    'Class': [class_]
                })
                topics_per_subClass_df = pd.concat([topics_per_subClass_df, new_row], ignore_index=True)
        
        return topics_per_subClass_df

    if filter:
        topics_per_class = topics_per_subclass()
    else:
        topics_per_class = bertopic_model.topics_per_class(df["processed_data"].astype(str).tolist(), classes=df[classes_column].to_list())

    if sortedBy=="Frequency":
        topics_per_class = topics_per_class.sort_values(by='Frequency', ascending=ascending)
    
    return topics_per_class

def visualize_topics_per_class_options(topic_model, topics_per_class, orient="h", **kwargs):
    """
    Visualize the topic representation of major topics per class.

    This function takes as input a dataframe `df`, a topic model `topic_model`, a column name `classes_column` representing the classes, an orientation `orient` which can be either horizontal ("h") or vertical ("v"), an optional parameter `sortedBy` which can be used to sort the topics by either "Frequency" or "Name", an optional parameter `ascending` which determines the sorting order (ascending or descending), and additional keyword arguments `**kwargs`.

    The function first calculates the topics per class using the `topics_per_class` method of the topic model, which takes as input the processed data from the dataframe and the classes. Then, depending on the specified orientation and sorting options, it either uses the `visualize_topics_per_class` method of the topic model (if the orientation is horizontal) or a modified version of this method called `visualize_topics_per_class2` (if the orientation is vertical) to create a figure representing the topics per class. The figure is then returned.

    This function allows you to easily visualize the distribution of topics per class in your data using a topic model.

    Parameters:
        df (pandas.DataFrame): The input dataframe containing the data to be visualized.
        topic_model (TopicModel): The topic model used to calculate the topics per class.
        classes_column (str): The name of the column in `df` representing the classes.
        orient (str): The orientation of the visualization. Can be either "h" for horizontal or "v" for vertical. Defaults to "h".
        sortedBy (str): An optional parameter used to sort the topics by either "Frequency" or "Name". Defaults to None.
        ascending (bool): An optional parameter used to determine the sorting order. If True, sorts in ascending order. If False, sorts in descending order. Defaults to True.
        **kwargs: Additional keyword arguments passed to the visualization method.

    Returns:
        matplotlib.figure.Figure: The figure representing the topics per class.
    """
    # Modify the visualize_topics_per_class method from bertopic to be able to print a barchart vertically
    def visualize_topics_per_class_orient(topic_model,
                                topics_per_class: pd.DataFrame,
                                top_n_topics: int = 10,
                                topics: List[int] = None,
                                normalize_frequency: bool = False,
                                custom_labels: Union[bool, str] = False,
                                title: str = "<b>Topics per Class</b>",
                                width: int = 1250,
                                height: int = 900,
                                orient: str = "h") -> go.Figure:
        """ Visualize topics per class

        Arguments:
            topic_model: A fitted BERTopic instance.
            topics_per_class: The topics you would like to be visualized with the
                            corresponding topic representation
            top_n_topics: To visualize the most frequent topics instead of all
            topics: Select which topics you would like to be visualized
            normalize_frequency: Whether to normalize each topic's frequency individually
            custom_labels: If bool, whether to use custom topic labels that were defined using 
                        `topic_model.set_topic_labels`.
                        If `str`, it uses labels from other aspects, e.g., "Aspect1".
            title: Title of the plot.
            width: The width of the figure.
            height: The height of the figure.
            orient: The orientation of the barchart, 'h' for horizontal, anything else for vertical

        Returns:
            A plotly.graph_objects.Figure including all traces

        Examples:

        To visualize the topics per class, simply run:

        ```python
        topics_per_class = topic_model.topics_per_class(docs, classes)
        topic_model.visualize_topics_per_class(topics_per_class)
        ```

        Or if you want to save the resulting figure:

        ```python
        fig = topic_model.visualize_topics_per_class(topics_per_class)
        fig.write_html("path/to/file.html")
        ```
        <iframe src="../../getting_started/visualization/topics_per_class.html"
        style="width:1400px; height: 1000px; border: 0px;""></iframe>
        """
        colors = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#D55E00", "#0072B2", "#CC79A7"]

        # Select topics based on top_n and topics args
        freq_df = topic_model.get_topic_freq()
        freq_df = freq_df.loc[freq_df.Topic != -1, :]
        if topics is not None:
            selected_topics = list(topics)
        elif top_n_topics is not None:
            selected_topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
        else:
            selected_topics = sorted(freq_df.Topic.to_list())

        # Prepare data
        if isinstance(custom_labels, str):
            topic_names = [[[str(topic), None]] + topic_model.topic_aspects_[custom_labels][topic] for topic in topics]
            topic_names = ["_".join([label[0] for label in labels[:4]]) for labels in topic_names]
            topic_names = [label if len(label) < 30 else label[:27] + "..." for label in topic_names]
            topic_names = {key: topic_names[index] for index, key in enumerate(topic_model.topic_labels_.keys())}
        elif topic_model.custom_labels_ is not None and custom_labels:
            topic_names = {key: topic_model.custom_labels_[key + topic_model._outliers] for key, _ in topic_model.topic_labels_.items()}
        else:
            topic_names = {key: value[:40] + "..." if len(value) > 40 else value
                        for key, value in topic_model.topic_labels_.items()}
        topics_per_class["Name"] = topics_per_class.Topic.map(topic_names)
        data = topics_per_class.loc[topics_per_class.Topic.isin(selected_topics), :]

        # Add traces
        fig = go.Figure()
        for index, topic in enumerate(selected_topics):
            if index == 0:
                visible = True
            else:
                visible = "legendonly"
            trace_data = data.loc[data.Topic == topic, :]
            topic_name = trace_data.Name.values[0]
            words = trace_data.Words.values
            if normalize_frequency:
                x = normalize(trace_data.Frequency.values.reshape(1, -1))[0]
            else:
                x = trace_data.Frequency

            ### Old part from the source github of bertopic ###
            # fig.add_trace(go.Bar(y=trace_data.Class,
            #                      x=x,
            #                      visible=visible,
            #                      marker_color=colors[index % 7],
            #                      hoverinfo="text",
            #                      name=topic_name,
            #                      orientation="h",
            #                      hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))
            
            fig.add_trace(go.Bar(y=trace_data.Class if orient == "h" else x,
                                x=x if orient == "h" else trace_data.Class,
                                visible=visible,
                                marker_color=colors[index % 7],
                                hoverinfo="text",
                                name=topic_name,
                                orientation=orient,
                                hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))

        # Styling of the visualization
        fig.update_xaxes(showgrid=True)
        fig.update_yaxes(showgrid=True)
        fig.update_layout(
            xaxis_title="Normalized Frequency" if normalize_frequency else "Frequency",
            yaxis_title="Class",
            title={
                'text': f"{title}",
                'y': .95,
                'x': 0.40,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': dict(
                    size=22,
                    color="Black")
            },
            template="simple_white",
            width=width,
            height=height,
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
                font_family="Rockwell"
            ),
            legend=dict(
                title="<b>Global Topic Representation",
            )
        )
        return fig

    if orient=="h":
        # using the source method to do it, without having the possibility to choose the orient (at the date of 09/2023)
        # we keep the use of the source method even if we could use only the new visualize_topics_per_class_orient. Because it permits to know and enjoy the modifications done in visualize_topics_per_class in the future by the owner of this source code
        fig = topic_model.visualize_topics_per_class(topics_per_class, **kwargs)
    else:  
        # using the modified source method to do it, adding the option to print the chart vertically
        fig = visualize_topics_per_class_orient(topic_model, topics_per_class, orient=orient, **kwargs)

    return fig

def create_chart_per_class(
                            df, 
                            bertopic_model, 
                            classes_column, 
                            filter=False, 
                            filter_group=None, 
                            filter_value=None, 
                            sortedBy=None, 
                            ascending=True, 
                            orient="h", 
                            **kwargs):
    """
    Create a chart representing the topics per class.

    This function takes as input a dataframe `df`, a BERTopic model `bertopic_model`, a column name `classes_column` representing the classes, an optional boolean parameter `filter` which determines whether to filter the data based on a subclass, an optional parameter `subclass_name` representing the column name of the subclass to filter on, an optional parameter `subclass_value` representing the value of the subclass to filter on, an optional parameter `sortedBy` which can be used to sort the topics by either "Frequency" or "Name", an optional boolean parameter `ascending` which determines the sorting order (ascending or descending), an orientation `orient` which can be either horizontal ("h") or vertical ("v"), and additional keyword arguments `**kwargs`.

    The function first calls the `create_topics_per_class_df` function to create a dataframe representing the topics per class. Then, it calls the `visualize_topics_per_class_options` function to create a chart from this dataframe. The resulting chart is then returned.

    Parameters:
        df (pandas.DataFrame): The input dataframe containing the data to be visualized.
        bertopic_model (BERTopic): The BERTopic model used to calculate the topics per class.
        classes_column (str): The name of the column in `df` representing the classes.
        filter (bool): An optional boolean parameter used to determine whether to filter the data based on a subclass. Defaults to False.
        filter_group (str): An optional parameter representing the column name of the subclass to filter on. Only used if `filter` is True. Defaults to None.
        filter_value: An optional parameter representing the value of the subclass to filter on. Only used if `filter` is True. Defaults to None.
        sortedBy (str): An optional parameter used to sort the topics by either "Frequency" or "Name". Defaults to None.
        ascending (bool): An optional boolean parameter used to determine the sorting order. If True, sorts in ascending order. If False, sorts in descending order. Defaults to True.
        orient (str): The orientation of the visualization. Can be either "h" for horizontal or "v" for vertical. Defaults to "h".
        **kwargs: Additional keyword arguments passed to the visualization method.

    Returns:
        plotly.graph_objs.Figure: The resulting chart representing the topics per class.
    """
    topics_per_class = create_topics_per_class_df(df, bertopic_model, classes_column, filter=filter, subclass_name=filter_group, subclass_value=filter_value, sortedBy=sortedBy, ascending=ascending)
    fig = visualize_topics_per_class_options(bertopic_model, topics_per_class, orient=orient, **kwargs)

    return fig 

def save_graph_html(fig, path, name):
    # Create the directory if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)
        
    return fig.write_html(path+"/"+name+".html")

In [159]:
### Graph barchart topics by sentiment

# Code without using the create_chart_per_class wrapper function
# create topics_per_class df in order to run the visualization then.
# topics_per_class = create_topics_per_class_df(df4, topic_model_merged, "sentiment_label", sortedBy="Frequency")
# visualize the topic representation of major topics per Sentiment (positive, negative, neutral):
# fig = visualize_topics_per_class_options(topic_model_merged, topics_per_class, orient="v", custom_labels=True, title="Topics per Sentiment", width=500, height=750)

# Code using the create_chart_per_class wrapper function
fig = create_chart_per_class(df4, topic_model_merged, "sentiment_label", 
                             sortedBy="Frequency", 
                             orient="v", 
                             custom_labels=True, 
                             title="Topics per Sentiment", 
                             width=500, height=750)

# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_merged_per_class_sentiment")
# fig

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [14]:
### Graph barchart topics by emotion

# visualize the topic representation of major topics per Emotions (28 different emotions):
fig = create_chart_per_class(df4, topic_model_merged, "single_emotion_label", 
                             custom_labels=True, 
                             title="Topics per Emotion", 
                             width=1150, height=750)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_merged_per_class_emotion")
# fig

In [16]:
### Graph barchart topics by year

# visualize the topic representation of major topics per Year:
fig = create_chart_per_class(df4, topic_model_merged, "year", 
                             custom_labels=True, 
                             title="Topics per Year", 
                             width=1400, height=750)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_merged_per_class_year")
# fig

In [18]:
### Graph barchart topics by "by_class_name" parameter for each value from "subclass_names"

# Create a dictionary that match column name to a shorter name for saving name file
shorter_names={
    "single_emotion_label":"emotion",
    "sentiment_label":"sentiment",
    "Market Segment":"market",
    "Account Country":"country",
    "Zone":"zone",
    "Clusters":"cluster",
    "year":"year"
}

subclass_name = "single_emotion_label"
subclass_name_shorter = shorter_names[subclass_name]
by_class_name = "year"
# Loop for each value from the subclass
for subclass_value in df4[subclass_name].unique() :
    # visualize the topic representation of major topics per by_class_name for each value from subclass_name:
    fig = create_chart_per_class(df4, topic_model_merged, by_class_name,
                                filter=True, 
                                subclass_name=subclass_name, subclass_value=subclass_value, 
                                sortedBy="Frequency",
                                custom_labels=True, 
                                title=f"Topics per {shorter_names[by_class_name]} - {subclass_name_shorter}: {subclass_value}", 
                                width=1400, height=750)

    # Save the figure as an HTML file
    path = f"../data/graphs/topics_per_class/topic_model_merged/per_{shorter_names[by_class_name]}/for_{subclass_name_shorter}"
    save_graph_html(fig, path, subclass_value)

In [ ]:
### Graph barchart for each topic with their top 5 keywords

# create barchart for each of the top_n_topics best topics
fig = topic_model.visualize_barchart(top_n_topics=12)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topics_barchart_viz")
fig

In [ ]:
### Graph hierarchical topics

bertopic_model = topic_model
hierarchical_topics = bertopic_model.hierarchical_topics(docs)
fig = bertopic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics, custom_labels=True, width=1200, height=950)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_hierarchy")
fig

In [ ]:
### Graph visualization of the documents in space

# Run the visualization with the original embeddings
topic_model_merged.visualize_documents(docs, embeddings=embeddings, custom_labels=True, hide_annotations=True)

In [20]:
### Graph visualization of the documents after reducing embeddings in space

# Reduce dimensionality of embeddings, this step is optional
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
fig=topic_model_merged.visualize_documents(docs, reduced_embeddings=reduced_embeddings, custom_labels=True, hide_annotations=True)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_merged_visualize_reduced_docs")
# fig

In [ ]:
### Graph visualization of the topics over time

bertopic_model = topic_model_merged
topics_over_time = bertopic_model.topics_over_time(docs, df4['year_month'])#, nr_bins=20)
fig = bertopic_model.visualize_topics_over_time(topics_over_time, custom_labels=True)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_merged_topics_over_time_months")
fig

In [ ]:
### Graph heatmap

fig = topic_model_merged.visualize_heatmap(custom_labels=True, n_clusters=4, height=600)
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_merged_heatmap")

fig2 = topic_model.visualize_heatmap(n_clusters=8,  height=750)
# Save the figure as an HTML file
save_graph_html(fig2, "../data/graphs", "topic_heatmap")

fig2

In [80]:
### Graph Visualize topics in 2D space

fig = topic_model.visualize_topics()
# Save the figure as an HTML file
save_graph_html(fig, "../data/graphs", "topic_visualize_topics")

fig2 = topic_model_merged.visualize_topics()
# Save the figure as an HTML file
save_graph_html(fig2, "../data/graphs", "topic_merged_visualize_topics")

fig2

In [83]:
### Graph Topic distribution for one specific document

topic_distr, topic_token_distr = topic_model_merged.approximate_distribution(docs, calculate_tokens=True)
topic_model_merged.visualize_distribution(topic_distr[1], custom_labels=True)

In [85]:
### Graph Topic distribution by keywords for one specific document

bertopic_model = topic_model_merged
topic_distr, topic_token_distr = bertopic_model.approximate_distribution(docs, calculate_tokens=True, window=4)
# # Visualize the token-level distributions
bertopic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])

In [ ]:
### Create the vocabulary using the OOP

# import nltk
# import sys
# sys.path.insert(0, '..')
# from utils import ngrams_list, more_stopwords, keybert_kwargs
# from vocabulary.vocabulary import VocabularyCreator
# from sklearn.feature_extraction.text import CountVectorizer

# nltk.download('stopwords')
# stopwords = nltk.corpus.stopwords.words('english')
# stopwords.extend(more_stopwords)

# vocabulary_creator = VocabularyCreator(ngrams_list, **keybert_kwargs)
# vocabulary_list = vocabulary_creator.keybert_vocabulary(df4)
# bertopic_vectorizer = CountVectorizer(vocabulary=vocabulary_list, stop_words=stopwords, lowercase=True, ngram_range=(1, 3))

In [22]:
### Wordcloud 
# Lemmatize keywords and recompute their probabilities for the wordcloud image

# Download required resources
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

def lemmatize_words(topic_words):
    """
    Lemmatize the words and combine their probabilities.

    This function takes as input a list of tuples `topic_words`, where each tuple contains a word and its probability. The function lemmatizes each word using the WordNetLemmatizer from the NLTK library, and combines the probabilities of the lemmas and their inflected forms.

    The resulting dictionary, where the keys are the lemmas and the values are their combined probabilities, is then returned.

    Parameters:
        topic_words (list): A list of tuples, where each tuple contains a word (str) and its probability (float).

    Returns:
        dict: A dictionary where the keys are the lemmas (str) and the values are their combined probabilities (float).
    """
    def get_wordnet_pos(treebank_tag):
        """
        Convert NLTK part of speech tags to WordNet tags.

        This function takes as input a part of speech tag in the format used by the NLTK library and returns the corresponding WordNet tag. The mapping between NLTK and WordNet tags is as follows:
        - 'J' (adjective) maps to `wordnet.ADJ`
        - 'V' (verb) maps to `wordnet.VERB`
        - 'N' (noun) maps to `wordnet.NOUN`
        - 'R' (adverb) maps to `wordnet.ADV`
        - All other tags map to `wordnet.NOUN`

        Parameters:
            treebank_tag (str): The NLTK part of speech tag to be converted.

        Returns:
            str: The corresponding WordNet part of speech tag.
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    # Create a lemmatizer object
    lemmatizer = WordNetLemmatizer()
    # Create a dictionary to store the lemmas and their probabilities
    lemma_prob = {}

    # Lemmatize each word and combine their probabilities
    for word, prob in topic_words:
        # Tokenize the word and get its part of speech
        tokens = nltk.word_tokenize(word)
        pos = nltk.pos_tag(tokens)[0][1]
        # Get the WordNet part of speech tag
        wordnet_pos = get_wordnet_pos(pos)
        # Lemmatize the word
        lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
        
        # Combine the probabilities of the lemma and its inflected forms
        if lemma in lemma_prob:
            lemma_prob[lemma] += prob
        else:
            lemma_prob[lemma] = prob
    
    return lemma_prob

def recalculate_probabilities(lemma_prob, docs, topic_model):
    """
    Recalculate the c-TF-IDF scores for the lemmas.

    This function takes as input a dictionary `lemma_prob` containing the lemmas and their probabilities, a list of documents `docs`, and a BERTopic model `topic_model`. The function recalculates the c-TF-IDF scores for the lemmas using the provided documents and BERTopic model.

    The function first calculates the term frequencies for each lemma using the CountVectorizer from the BERTopic model. Then, it calculates the inverse document frequencies for each lemma and uses these values to compute the c-TF-IDF scores. The c-TF-IDF scores are then normalized and used to update the probabilities of the lemmas.

    The resulting dictionary, where the keys are the lemmas and the values are their updated probabilities, is then returned.

    Parameters:
        lemma_prob (dict): A dictionary where the keys are the lemmas (str) and the values are their probabilities (float).
        docs (list): A list of documents used to fit the BERTopic model.
        topic_model (BERTopic): The BERTopic model used to calculate the c-TF-IDF scores.

    Returns:
        dict: A dictionary where the keys are the lemmas (str) and the values are their updated probabilities (float).
    """
    # Calculate the term frequencies using the provided CountVectorizer
    X = topic_model.vectorizer_model.transform(docs)
    
    # Calculate the term frequencies for each lemma
    tf = {}
    for lemma, prob in lemma_prob.items():
        index = topic_model.vectorizer_model.vocabulary_.get(lemma)
        if index is not None:
            tf[lemma] = np.sum(X[:, index])

    # Calculate the inverse document frequencies for each lemma
    df = np.sum(X > 0, axis=0)
    N = X.shape[0]
    idf = np.log(N / (df + 1))

    # Calculate the c-TF-IDF scores for each lemma and normalize them
    c_tf_idf = {}
    for lemma, prob in lemma_prob.items():
        index = topic_model.vectorizer_model.vocabulary_.get(lemma)
        if index is not None:
            c_tf_idf[lemma] = tf[lemma] * idf[0, index]
    c_tf_idf_sum = np.sum(list(c_tf_idf.values()))
    for lemma, score in c_tf_idf.items():
        c_tf_idf[lemma] /= c_tf_idf_sum

    # Update the probabilities of the lemmas based on their c-TF-IDF scores
    new_lemma_prob = {}
    for lemma, prob in lemma_prob.items():
        if lemma in c_tf_idf:
            new_lemma_prob[lemma] = c_tf_idf[lemma]
    
    return new_lemma_prob

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cattiaux/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/cattiaux/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/cattiaux/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
### Wordcloud

def get_topic_words(bertopic_model, topic, top_n=10):
    """
    Get the top n words for a given topic.

    This function takes as input a BERTopic model `topic_model`, a topic number `topic`, and an optional integer parameter `top_n` specifying the number of words to return. The function returns the top n words for the given topic, along with their probabilities, as a list of tuples.

    The function first retrieves the c-TF-IDF matrix and feature names from the BERTopic model. Then, it gets the row of the c-TF-IDF matrix corresponding to the given topic and uses it to find the indices of the top n words. Finally, it retrieves the words and their probabilities and returns them as a list of tuples.

    Parameters:
        topic_model (BERTopic): The BERTopic model used to calculate the topic words.
        topic (int): The topic number for which to retrieve the top n words.
        top_n (int): An optional integer parameter specifying the number of words to return. Defaults to 10.

    Returns:
        list: A list of tuples, where each tuple contains a word (str) and its probability (float).
    """
    # Get the c-TF-IDF matrix and feature names
    c_tf_idf = bertopic_model.c_tf_idf_.toarray()
    feature_names = bertopic_model.vectorizer_model.get_feature_names_out()
    
    # Get the row of the c-TF-IDF matrix corresponding to the topic
    topic_row = c_tf_idf[topic]
    
    # Get the indices of the top_n words for the topic
    top_n_indices = np.argsort(topic_row)[::-1][:top_n]
    
    # Get the words and their probabilities
    words = [feature_names[i] for i in top_n_indices]
    probabilities = [topic_row[i] for i in top_n_indices]
    
    # Return the words and their probabilities as a list of tuples
    return list(zip(words, probabilities))

def group_docs_by_topic(docs, bertopic_model):
    """
    Group documents by their assigned topic.

    This function takes as input a list of documents `docs` and a BERTopic model `bertopic_model`. It creates a dictionary where the keys are topic numbers and the values are lists of documents assigned to each topic.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The BERTopic model used to assign topics to the documents.

    Returns:
        dict: A dictionary where the keys are topic numbers (int) and the values are lists of documents (list) assigned to each topic.
    """
    docs_by_topic = {}
    for doc, topic in zip(docs, bertopic_model.topics_):
        if topic+1 not in docs_by_topic:
            docs_by_topic[topic+1] = []
        docs_by_topic[topic+1].append(doc)
    
    return docs_by_topic 

def get_word_freq(bertopic_model, docs, topic, top_n=10, scale=1, lemmatize=False):
    """
    Get the word frequencies for a given topic.

    This function takes as input a BERTopic model `bertopic_model`, a list of documents `docs`, a topic number `topic`, an optional integer parameter `top_n` specifying the number of words to include, an optional float parameter `scale` used to scale the probabilities of the words and an optional boolean parameter `lemmatize` which determines whether to lemmatize the words before calculating their frequencies.

    The function first retrieves the top n words for the given topic using the `get_topic_words` function and scales their probabilities using the provided `scale` parameter. If `lemmatize` is `True`, the function lemmatizes the words using the `lemmatize_words` function and recalculates their probabilities using the `recalculate_probabilities` function. Otherwise, it uses the original words and their probabilities.

    Parameters:
        bertopic_model (BERTopic): The BERTopic model used to calculate the topic words.
        docs (list): A list of documents used to fit the BERTopic model.
        topic (int): The topic number for which to calculate the word frequencies.
        top_n (int): An optional integer parameter specifying the number of words to include. Defaults to 10.
        scale (float): An optional float parameter used to scale the probabilities of the words. Defaults to 1.
        lemmatize (bool): An optional boolean parameter used to determine whether to lemmatize the words before calculating their frequencies. Defaults to False.

    Returns:
        dict: A dictionary where the keys are the words/lemmas (str) and the values are their probabilities (float).
    """
    # Get the topic words and their probabilities
    topic_words = get_topic_words(bertopic_model, topic, top_n=top_n)
    # Scale the probabilities
    topic_words = [(word, prob ** scale) for word, prob in topic_words]

    if lemmatize:
        # Group documents by their assigned topic.
        docs_by_topic = group_docs_by_topic(docs, bertopic_model)
        # get the documents assigned to a specific topic
        my_docs = docs_by_topic.get(topic, [])
        # Lemmatize the words and combine their probabilities
        lemma_prob = lemmatize_words(topic_words)
        # Recalculate the c-TF-IDF scores for the lemmas
        topic_words_lemma = recalculate_probabilities(lemma_prob, my_docs, bertopic_model)
        # Create a dictionary with the lemmas and their probabilities
        word_freq = {lemma: prob for lemma, prob in topic_words_lemma.items()}
    
    else:
        # Create a dictionary with the words and their probabilities
        word_freq = {word: prob for word, prob in topic_words}
    
    return word_freq 

def create_wordcloud(bertopic_model, docs, topic, top_n=10, scale=1, lemmatize=False, stopwords=None, wordcloud_kwargs=None):
    """
    Create a word cloud from a BERTopic model and a topic.

    This function takes as input a BERTopic model `topic_model`, a list of documents `docs`, a topic number `topic`, an optional integer parameter `top_n` specifying the number of words to include in the word cloud, an optional float parameter `scale` used to scale the probabilities of the words, an optional boolean parameter `lemmatize` which determines whether to lemmatize the words before creating the word cloud, an optional list of stopwords `stopwords` to be removed from the word cloud, and an optional dictionary of keyword arguments `wordcloud_kwargs` to be passed to the WordCloud constructor.

    The function first retrieves the top n words for the given topic using the `get_topic_words` function and scales their probabilities using the provided `scale` parameter. If `lemmatize` is `True`, the function lemmatizes the words using the `lemmatize_words` function and recalculates their probabilities using the `recalculate_probabilities` function. Otherwise, it uses the original words and their probabilities.

    The function then removes any stopwords from the list of words (if provided) and creates a word cloud using the WordCloud class from the wordcloud library. The resulting word cloud is then returned.

    Parameters:
        topic_model (BERTopic): The BERTopic model used to calculate the topic words.
        docs (list): A list of documents used to fit the BERTopic model.
        topic (int): The topic number for which to create the word cloud.
        top_n (int): An optional integer parameter specifying the number of words to include in the word cloud. Defaults to 10.
        scale (float): An optional float parameter used to scale the probabilities of the words. Defaults to 1.
        lemmatize (bool): An optional boolean parameter used to determine whether to lemmatize the words before creating the word cloud. Defaults to False.
        stopwords (list): An optional list of stopwords to be removed from the word cloud. Defaults to None.
        wordcloud_kwargs (dict): An optional dictionary of keyword arguments to be passed to the WordCloud constructor. Defaults to None.

    Returns:
        wordcloud.WordCloud: The resulting word cloud.
    """
    # Get the word frequencies for a given topic.
    word_freq = get_word_freq(bertopic_model, docs, topic, top_n=top_n, scale=scale, lemmatize=lemmatize)

    # Remove stopwords from word_freq if provided
    if stopwords:
        word_freq = {word: freq for word, freq in word_freq.items() if word not in stopwords}
    
    # Create the word cloud using the words/lemmas and their probabilities
    wc = WordCloud(**(wordcloud_kwargs or {}))
    wc.generate_from_frequencies(word_freq)

    # Display the word cloud
    # plt.imshow(wc, interpolation='bilinear')
    # plt.axis("off")
    # plt.show()
    return wc

def create_wordclouds_bertopic(bertopic_model, docs, top_n=10, scale=1, lemmatize=False, stopwords=None, wordcloud_kwargs=None, to_save=False, save_path=None):
    """
    Create word clouds for all topics in a BERTopic model.

    This function takes as input a BERTopic model `bertopic_model`, a list of documents `docs`, an optional integer parameter `top_n` specifying the number of words to include in each word cloud, an optional float parameter `scale` used to scale the probabilities of the words, an optional boolean parameter `lemmatize` which determines whether to lemmatize the words before creating the word clouds, an optional list of stopwords `stopwords` to be removed from the word clouds, an optional dictionary of keyword arguments `wordcloud_kwargs` to be passed to the WordCloud constructor, an optional boolean parameter `to_save` which determines whether to save the word clouds as image files, and an optional string parameter `save_path` specifying the path where the image files should be saved.

    The function first retrieves the topic information from the BERTopic model and sets the index of the resulting DataFrame to be the topic number. Then, it loops over the topic numbers in the DataFrame and calls the `create_wordcloud` function to create a word cloud for each topic. If `to_save` is `True`, it saves each word cloud as an image file at the specified location using the custom name of the topic.

    The resulting dictionary, where the keys are the custom names of the topics and the values are their corresponding word clouds, is then returned.

    Parameters:
        bertopic_model (BERTopic): The BERTopic model used to calculate the topic words.
        docs (list): A list of documents used to fit the BERTopic model.
        top_n (int): An optional integer parameter specifying the number of words to include in each word cloud. Defaults to 10.
        scale (float): An optional float parameter used to scale the probabilities of the words. Defaults to 1.
        lemmatize (bool): An optional boolean parameter used to determine whether to lemmatize the words before creating the word clouds. Defaults to False.
        stopwords (list): An optional list of stopwords to be removed from each word cloud. Defaults to None.
        wordcloud_kwargs (dict): An optional dictionary of keyword arguments to be passed to each WordCloud constructor. Defaults to None.
        to_save (bool): An optional boolean parameter used to determine whether to save each word cloud as an image file. Defaults to False.
        save_path (str): An optional string parameter specifying the path where each image file should be saved. Only used if `to_save` is True. Defaults to None.

    Returns:
        dict: A dictionary where the keys are the custom names of the topics (str) and the values are their corresponding word clouds (wordcloud.WordCloud).
    """
    # Check that save_path is provided if to_save is True
    if to_save and save_path is None:
        raise ValueError("If to_save is True, save_path must be provided")
    
    # Get the topic information
    topic_info = bertopic_model.get_topic_info()
    # Set the index of the DataFrame to be the topic number
    topic_info = topic_info.set_index('Topic')
    wc_pics={}
    # Loop over the topic numbers in the DataFrame
    for topic_number in topic_info.index:
        # Get the custom name of the current topic for saving purpose
        topic_custom_name = topic_info.loc[topic_number, 'CustomName']
        wc_pic = create_wordcloud(bertopic_model, docs, topic_number+1, top_n=top_n, scale=scale, lemmatize=lemmatize, stopwords=stopwords, wordcloud_kwargs=wordcloud_kwargs)
        wc_pics.update({topic_custom_name: wc_pic})

        if to_save and lemmatize:
            wc_pic.to_file(f'{save_path}/{topic_custom_name}_lemmatized.png')
        elif to_save and lemmatize==False:
            wc_pic.to_file(f'{save_path}/{topic_custom_name}.png')

    return wc_pics

# examples of default colors for the wordcloud
# colormaps = [
#     'viridis', 'plasma', 'inferno', 'magma', 'cividis',
#     'Greys', 'Purples', 'Blues', 'Greens', 'Oranges',
#     'Reds', 'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd',
#     'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu',
#     'PuBuGn', 'BuGn', 'YlGn'
# ]

# Create a custom colormap
# colors = ["#294D61", "#6DA5C0", "#0F969C", "#0C7075", "#072E33","#05161A"]
colors = ["#F1916D", "#F5D7DB", "#BD83B8", "#473E66", "#1B3358","#06142E"]
my_cmap = ListedColormap(colors)

# Create a binary mask from an image
# make sure that the image is black and white, where black pixels indicate where to draw words and white pixels indicate where not to draw words.
# mask = np.array(Image.open('circle.png'))

wordcloud_stopwords = ["get","would","us","good","like","goods","got","able","quite","always","nothing","add","everything","think","gave","due","find","say","took","still","within","22","10","one","makhloufi","saliger","kessler","naldrin","hidde","mohua","possible","since","could","especially","altivar","every","anyway","egawa","sometimes"]

wordcloud_kwargs = {
    'width': 800,
    'height': 400,
    'background_color': 'black',
    'contour_width': 100,
    'contour_color': 'red',
    'colormap': 'GnBu',
    "contour_width": 5,
    "contour_color": 'red'
    # "mask":mask
}

wordclouds = create_wordclouds_bertopic(topic_model_merged, docs, 
                           top_n=50, scale=1, 
                           lemmatize=True, 
                           stopwords=wordcloud_stopwords, 
                           wordcloud_kwargs=wordcloud_kwargs,
                           to_save=True, 
                           save_path="../data/wordclouds/test2")

In [45]:
# Barchart from seaborn : emotions by year

# # Create a count plot
# sns.set(style="whitegrid")
# g = sns.catplot(x="year", y=None, hue="single_emotion_label", data=df4, kind="count", height=4, aspect=2)

# # Set the axis labels
# g.set_axis_labels("Year", "Count")

# # Set the title
# plt.title("Count of Emotions by Year")

# # Show the plot
# plt.show()

In [69]:
# Graph avec plotly.express : barchart for one defined emotion

# import ipywidgets as widgets
import plotly.express as px

def plot_emotion(df, emotion):
    """
    Plot a histogram of the distribution of a given emotion by year.

    This function takes as input a dataframe `df`, an emotion `emotion` and creates a histogram showing the distribution of this emotion by year. The data is filtered to only include rows with the specified emotion, and a histogram is created using the Plotly Express library. The axis labels and title are set, and the resulting plot is shown.

    Parameters:
        df (pandas.DataFrame): The input dataframe containing the data to be plotted.
        emotion (str): The emotion to be plotted.

    Returns:
        plotly.graph_objs.Figure: The resulting histogram plot.
    """
    # Filter the data to only include rows with the specified emotion
    filtered_data = df[df['single_emotion_label'] == emotion]
    
    # Create a histogram
    fig = px.histogram(filtered_data, x='year', nbins=20)
    
    # Set the axis labels
    fig.update_layout(xaxis_title='Year', yaxis_title='Count')
    
    # Set the title
    fig.update_layout(title=f'Distribution of {emotion} by Year')
    
    return fig

# # Call the function with the desired emotion
plot_emotion(df4, 'disappointment')

## Tests graph précédent avec menu déroulant pour choisir son emotion

# # Create a dropdown menu with all unique emotions in 'single_emotion_label' column
# emotions = df4['single_emotion_label'].unique()
# dropdown = widgets.Dropdown(options=emotions)

# # Create an output widget to display the plot
# output = widgets.Output()

# # Define a function that updates the plot when a new emotion is selected
# def on_change(change):
#     if change['name'] == 'value':
#         with output:
#             output.clear_output()
#             plot_emotion(change['new'])

# # Register the function as a callback for changes in the dropdown value
# dropdown.observe(on_change)

# # # Display the widgets
# display(dropdown)
# display(output)

In [88]:
### Graph barchart of one given emotion by a specified class (year, zone, etc.)

# Plot a bar chart or histogram of the distribution of a given emotion by a specified class
def plot_emotion(df, emotion, class_name, time_period=None, random_colors=True, set_colors=['#96ceb4', '#87bdd8', '#ffcc5c', '#ff6f69', '#f4a688', '#d96459'], set_color=None):
    """
    Plot a bar chart or histogram of the distribution of a given emotion by a specified class.

    This function takes as input a dataframe `df`, an emotion `emotion`, a class name `class_name` representing the column by which to group the data, an optional time period `time_period` specifying the year to filter the data by, an optional boolean parameter `random_colors` which determines whether to use a random color for the plot or a specified color, an optional list of colors `set_colors` to choose from if `random_colors` is `True`, and an optional color `set_color` to use if `random_colors` is `False`.

    The data is filtered to only include rows with the specified emotion, and further filtered by the specified time period if provided. If `random_colors` is `True`, a random color is chosen from the provided list of colors. Otherwise, the specified color is used.

    If `class_name` is not 'year', the function creates a bar chart showing the distribution of the emotion by the specified class. Otherwise, it creates a histogram showing the distribution of the emotion by year. The axis labels and title are set, and the resulting plot is returned.

    Parameters:
        df (pandas.DataFrame): The input dataframe containing the data to be plotted.
        emotion (str): The emotion to be plotted.
        class_name (str): The name of the column in `df` representing the class by which to group the data.
        time_period (int): An optional integer parameter specifying the year to filter the data by. Only used if `class_name` is not 'year'. Defaults to None.
        random_colors (bool): An optional boolean parameter used to determine whether to use a random color for the plot or a specified color. Defaults to True.
        set_colors (list): An optional list of colors to choose from if `random_colors` is True. Defaults to ['#96ceb4', '#87bdd8', '#ffcc5c', '#ff6f69', '#f4a688', '#d96459'].
        set_color (str): An optional color to use if `random_colors` is False. Defaults to None.

    Returns:
        plotly.graph_objs.Figure: The resulting bar chart or histogram plot.
    """
    # Check that set_color is provided if random_colors is False
    if random_colors==False and set_color is None:
        raise ValueError("set_color must be provided if random_colors is False")
    
    # Filter the data to only include rows with the specified emotion
    filtered_data = df[df['single_emotion_label'] == emotion]

    # Filter the data by the specified time period if provided
    if time_period != None:
        if class_name == 'year':
            raise ValueError("class_name cannot be 'year' when time_period is defined")
        filtered_data = filtered_data[filtered_data['year'] == time_period]

    if random_colors:
        # Randomly choose a color from the defined colors list
        colors_list = set_colors
        color = random.choice(colors_list)
    else:
        color = set_color

    # Sort the data by ascending frequency if class_name is not 'year'
    if class_name != 'year':
        filtered_data = filtered_data[class_name].value_counts(ascending=True).reset_index()
        filtered_data.columns = [class_name, 'count']
        fig = px.bar(filtered_data, x=class_name, y='count', color_discrete_sequence=[color], width=1100, height=600)
    else:
        # Create a histogram
        fig = px.histogram(filtered_data, x=class_name, nbins=20, color_discrete_sequence=[color], width=1100, height=600)

    # Set the axis labels
    fig.update_layout(xaxis_title=class_name, yaxis_title='Count')
    # Set the title
    fig.update_layout(title=f'Distribution of {emotion} by {class_name}' + (f' in {time_period}' if time_period is not None else ''))
    # Center the title
    fig.update_layout(title_x=0.5)

    return fig

emotions = df4['single_emotion_label'].unique()
class_name = "Account Country"
time_period = 2023
# Loop over each emotion and save the corresponding graph
for emotion in emotions:
    fig = plot_emotion(df4, emotion, class_name, time_period)
    # Export the chart as a PNG image
    pio.write_image(fig, f'../data/graphs/test/emotions/{emotion}_by_{class_name}_{time_period}.png')

# plot_emotion(df4, "love", "Account Country", time_period=2023)

In [156]:

def get_top_topic_docs(probabilities, df, topic, top_n_docs=len(df)):
    """
    Get the top n documents for a specified topic.

    Parameters:
    probabilities (numpy.ndarray): The probabilities from the BERTopic model.
    df (pandas.DataFrame): The dataframe containing the documents.
    topic (int): The topic to get the top n documents for.
    top_n_docs (int): The number of top documents to return. Defaults to the length of df.

    Returns:
    tuple: A tuple containing a dataframe with the top n documents for the specified topic and a list with their content.
    """
    def sort_docs_with_same_prob(probabilities, prob, topic):
        """
        Sort documents that have the same probability for a specified topic.

        Parameters:
        probabilities (numpy.ndarray): The probabilities from the BERTopic model.
        prob (float): The probability to filter by.
        topic (int): The topic to sort the documents for.

        Returns:
        numpy.ndarray: An array containing the sorted indices of the documents that have the specified probability for the specified topic.
        """
        # Get the indices of all documents that have the current probability for the defined topic
        top_docs_indices = np.where(probabilities[:, topic] == prob)[0]
        # Sort the probabilities for each document in descending order
        sorted_probs = -np.sort(-probabilities[top_docs_indices], axis=1)
        # Compute the score for each document based on the difference between their first and second scores
        scores = sorted_probs[:, 0] - sorted_probs[:, 1]
        # Sort these top documents based on their scores
        sorted_top_docs_indices = top_docs_indices[np.argsort(scores)[::-1]]
        
        return sorted_top_docs_indices
    
    # Get the unique probabilities for the defined topic
    unique_probs = np.unique(probabilities[:, topic])
    # Sort the unique probabilities in descending order
    sorted_unique_probs = np.sort(unique_probs)[::-1]
    
    # Initialize an empty list to store the sorted indices of all documents
    sorted_docs_indices = []
    
    # Loop over the unique probabilities
    for prob in sorted_unique_probs:
        # Sort the documents that have the current probability for the defined topic
        sorted_top_docs_indices = sort_docs_with_same_prob(probabilities, prob, topic)
        # Append these indices to the list of sorted indices of all documents
        sorted_docs_indices.extend(sorted_top_docs_indices)
    
    # Take the top n from this sorted list
    final_top_docs_indices = np.array(sorted_docs_indices)[:top_n_docs]
    
    # Get the content of the top n documents from your dataframe
    top_docs_df = df.iloc[final_top_docs_indices]
    top_docs_content = top_docs_df["processed_data"].to_list()
    
    return top_docs_df, top_docs_content

def filter_docs(df, filter_column, filter_value):
    """
    Filter a dataframe based on a specified column and value.

    Parameters:
    df (pandas.DataFrame): The dataframe to filter.
    filter_column (str): The name of the column to filter by.
    filter_value (str): The value to filter by in the specified column.

    Returns:
    tuple: A tuple containing the filtered dataframe and a boolean mask indicating which rows match the specified filter.
    """
    filter_mask = df[filter_column] == filter_value
    return df[filter_mask], filter_mask

# Filter the top docs based on the specified column and value
filtered_top_docs, filter_mask = filter_docs(df4, "Account Country", "France")
top_docs_df, top_docs_content = get_top_topic_docs(topic_model_merged.probabilities_[filter_mask], filtered_top_docs, topic=2, top_n_docs=10)

top_docs_content

['systematic ar + deadlines respected',
 'deadlines not respected and uncertain, no reliable information',
 'the order is not completely delivered',
 'correct and on delivery time',
 'sending a "deadline" satisfaction questionnaire when you are late for several months is a matter of psychiatry. opening case is useless. you make delivery commitments that you don\'t keep. you close the cases without notifying me. the manual allocations are allocated to the largest.',
 'the impossibility of having deadlines on the articles in allocation. deadline respected',
 "catastrophic management. i'm still not delivered",
 'on 25/10/21 mr verollet explains that a 1st part is carried out the second part will take place at the end of october at the beginning of november prefers reminder mid-november in order to take stock of the service in its entirety./ on 14/12/21 mr verollet gives 9/10 for the 2 parties for the availability of the teams and the skills of the people who intervened, totally adapted to